In [46]:
# Modules
import numpy as np
import xarray as xr
import pandas as pd

from data import data_loader
from data import gesla_preprocessing
from data import era5_preprocessing
from data import preprocessing

In [49]:
# ---
# Preprocessing
# ---

# Get timeseries of predictor and predictand
percentile = 0.95
predictor = "sp"
season = "autumn"
x1, y1, t1 = preprocessing.preprocessing1(season, predictor, percentile)

Load ERA5-Predictor: sp in region: lon-0530_lat7040 for years: 1999-2008 in season: autumn
Load Predictand from GESLA
Applied one-hot-encoding
Get overlapping timeseries of ERA5 and GESLA


(910, 2)

In [50]:
# Handle NaN values
y1.shape
# Prepare shape for model


(910, 2)

In [ ]:
#---
# Train Model
#---

In [ ]:
# Forecast

In [ ]:
# Postprocessing

In [ ]:
# Diagnostics

In [ ]:
# Visualization